<a href="https://colab.research.google.com/github/Rachna0604/Devrev/blob/main/AnotherApproachDevrev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.0 MB/s 
     |████████████████████████████████| 182 kB 43.6 MB/s 
     |████████████████████████████████| 7.6 MB 54.9 MB/s 


In [2]:
# accessing g drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


load the data from the CSV file into a Pandas DataFrame:

In [3]:
import pandas as pd

# Load the data from the CSV file
df = pd.read_csv('/content/drive/MyDrive/DevRev/train_data.csv')
df.drop(columns = df.columns[0], axis = 1, inplace= True)
df.pop('Answer_possible')
df.pop('Answer_text')
df.pop('Answer_start')

# Print the first few rows of the DataFrame
df.head()

,Theme,Paragraph,Question
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?


The DataFrame should have three columns: 'theme', 'question', and 'paragraph'.

Next, we need to split the data into training and validation sets

In [4]:
# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into training and validation sets
train_df = df[:int(len(df) * 0.8)]
val_df = df[int(len(df) * 0.8):]


define a Dataset class to load and preprocess the data. The Dataset class should take a DataFrame as input and return a tuple of question and paragraph tensors:

In [5]:
from torch.utils.data import Dataset
import transformers

class ParagraphDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get the question and paragraph text
        question = self.df.loc[idx, 'Question']
        paragraph = self.df.loc[idx, 'Paragraph']

        # Encode the text
        inputs = self.tokenizer.encode_plus(
            question,
            paragraph,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_tensors='pt'
        )

        # Return the encoded text as a tuple
        return (inputs['input_ids'], inputs['attention_mask'])


define a DataLoader to load the data in batches:

In [6]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Set the maximum sequence length
MAX_LEN = 512

# Create a DistilBERT tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-cased')

# Create the Dataset and DataLoader for the training set
train_dataset = ParagraphDataset(train_df, tokenizer, MAX_LEN)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create the Dataset and DataLoader for the validation set
val_dataset = ParagraphDataset(val_df, tokenizer, MAX_LEN)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

The batch_size parameter determines the number of samples per batch, and the shuffle parameter determines whether the data should be shuffled before each epoch.

we can then use the DataLoader to iterate over the data in batches by using a for loop:

In [7]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Initialize the DistilBert tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)
s= 0
t = 0
#input= inputs
for epoch in range(3):
    print('t', t)
    for batch in train_loader:
        if s%50 == 0:
          print(s)
        optim.zero_grad()
        #input=inputs.values()
        input_ids = batch['inputs'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        s+=1
    t+=1
model.eval()

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.b

t 0
0


TypeError: ignored

iteration over the training set

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Iterate over the training set in batches
for inputs, masks in train_dataloader:
    # Do something with the inputs and masks
    # For example, you could pass them to a model for training:
    outputs = model(inputs, attention_mask=masks)

    # Calculate the loss and backpropagate the gradients
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

iterate over the validation set in the same way:

In [ ]:
# Iterate over the validation set in batches
for inputs, masks in val_dataloader:
    # Do something with the inputs and masks
    # For example, we could pass them to a model for evaluation:
    outputs = model(inputs, attention_mask=masks)

    # Calculate the loss and backpropagate the gradients
    loss = criterion(outputs, labels)


use the outputs of the model to make predictions, calculate evaluation metrics, or do any other task 

In [ ]:
# Make predictions using the model
predictions = torch.argmax(outputs, dim=1)

# Calculate evaluation metrics
accuracy = (predictions == labels).float().mean()

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')


This code assumes that we have defined a labels tensor containing the ground truth labels for the inputs.

calculating other evaluation metrics, such as precision, recall, and F1 score.

In [ ]:
# Calculate precision, recall, and F1 score
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
f1 = f1_score(labels, predictions)

# Print the evaluation metrics
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 score: {f1:.4f}')


predict

In [ ]:
def predict(model, tokenizer, question, paragraph):
    # Tokenize the question and paragraph
    question_tokens = tokenizer.encode(question, add_special_tokens=True)
    paragraph_tokens = tokenizer.encode(paragraph, add_special_tokens=True)

    # Create the input array by concatenating the question and paragraph tokens
    input_tokens = question_tokens + paragraph_tokens[1:]
    input_ids = torch.tensor(input_tokens).unsqueeze(0)  # Add a batch dimension

    # Create the attention mask by marking the question and paragraph tokens as 1 and padding tokens as 0
    attention_mask = [1] * len(input_tokens)
    while len(attention_mask) < 512:  # Change 512 to the desired length
        attention_mask.append(0)
    attention_mask = torch.tensor(attention_mask).unsqueeze(0)  # Add a batch dimension

    # Set the model to evaluation mode
    model.eval()

    # Run the model on the input data and get the output logits
    logits = model(input_ids, attention_mask=attention_mask)[0]

    # Get the predicted label
    _, pred = torch.max(logits, dim=1)

    return pred

# Test the predict function
prediction = predict(model, tokenizer, "What is machine learning?", "Machine learning is a method of teaching computers to learn from data without explicitly programming them.")
print(f"Prediction: {prediction}")